In [1]:
import sys#TylerK: Add sys to get cmdline_helper from NRPy top directory; remove this line and next when debugged
sys.path.append('../')

import cmdline_helper as cmd     # NRPy+: Multi-platform Python command-line interface

# Create C code output directory:
Ccodesdir = "SEOBNR_Playground_Pycodes"

import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate as spint
import Hamiltonian.Hreal_and_csi_on_bottom as hr
import Derivatives.ham_and_csi_first_derivs as dh
import Flux.flux as fl
import Flux.factorized_waveform_for_flux as wf


# Tutorial - Right-Hand-Side's for Hamilton's Equation of Motion for SEOBNR Models

In this notebook, we will implement the right hand side functions for the Hamilton's EOMs so that we can integrate them to get the trajectory. Since we are using scipy's ODE integration functionality, we will be writing these as wrappers that will ultimately be passed into the scipy functions.

To begin, we will recap the ODEs that form the Hamilton's EOMs and document the individual terms. We then create the wrapper function that will pass the phase space variables and parameters to the derivative functions generated before.

## Step : Hamilton's EOMs for SEOBNR implementations

The Hamilton's EOMs for a general SEOBNR implementation is given in https://arxiv.org/pdf/0912.3466.pdf as a set of derivatives for the positions, momenta and spins.
We begin by stating the positional derivatives as

$$
\frac{dX^i}{dt} = T^i_j \frac{\partial \mathcal{H}_\rm{real}}{\partial P^*_j}
$$

where $P^*$ represent the tortoise momenta and the matrix $\mathbb{T}$ represents the transformation matrix from the tortoise momentum to the conjugate momentum. The momentum derivatives, on the other hand, are given as

$$
\frac{dP^*_i}{dt} = -T_i^j\frac{\partial \mathcal{H}_\rm{real}}{\partial X^j} + \frac{1}{\Omega L }\frac{dE}{dt}P^*_j + P^*_l\left(T^{-1}\right)_k^l\frac{\partial T_i^k}{\partial X^j}T_m^j\frac{\partial \mathcal{H}_\rm{real}}{\partial P^*_m} 
$$

where $\Omega$ is the rotational frequency, $L$ is the magnitude of the angular momentum and $\frac{dE}{dt}$ is the energy flux due ot gravitational waves. Since we already have dedicate functions to compute the Hamiltonian derivates and the fluxes, in this notebook we will document the terms unique to the RHS computations and call the dedicated functions as necessary.

Finally, the spin derivatives are given as (where $a$ labels the individual spin vectors so $a \in \{1,2\}$)

$$
\dot{\bf{S}}\bf{_a} = \{\bf{S_a},\mathcal{H}\} = \frac{\partial \mathcal{H}}{\partial \bf{S_a}}\times\bf{S_a}
$$


### The Angular Momentum Magnitude $L$

The magnitude of the angular momentum vector is defined as follows

$$
L = \sqrt{\bf{L}\cdot\bf{L}}
$$

### The Angular Momentum Vector $\bf{L}$

The angular momentum vector has its usual definition and is defined using the tortoise momentum due to (Eq.A1 of PRD 81, 084041 (2010))

$$
\bf{L} = \bf{X}\times\bf{P} = \bf{X}\times\bf{P^*}
$$

### The Rotational Frequence $\Omega$

The rotational frequency is defined here as

$$
\Omega = \frac{1}{R^2}\sqrt{\bf{X}\times\dot{\bf{X}}}
$$

Where $R$ is the radial distance and $\dot{\bf{X}}$ is computed as the first set of right hand sides.


### The Positional Derivatives $\dot{\bf{X}}$

The derivatives of the coordinates are given as:

$$
\frac{dX^i}{dt} = T^i_j \frac{\partial \mathcal{H}_\rm{real}}{\partial P^*_j}
$$

### The $\mathbb{T}$ matrix

The trnasformation matrix from tortoise to conjugate momentum are given as

$$
T_i^j = \delta_{ij} + \frac{X^iX^j}{R^2}\left[\xi_a - 1\right]
$$

while the inverse matrix is given as

$$
\left(T^{-1}\right)_i^j = \delta_{ij} - \frac{\xi_a - 1}{\xi_a}\frac{X^iX^j}{R^2} 
$$

and the derivative of the matrix is given as

$$
\frac{\partial T_i^j}{\partial X^k} = \frac{\delta_{ik}X^j + X^i\delta_{jk}}{R^2}\left(\xi_a - 1\right) - 2\frac{X^iX^jX^k}{R^4}\left(\xi_a - 1\right) + \frac{X^iX^j}{R^2}\frac{\partial\xi_a}{\partial X^k} 
$$

Where, $\xi_a$ is the tortoise factor defined below and $R$ is the radial distance

### The Radial Distance $R$

The radial distance $R$ is given as usual

$$
R = \sqrt{\bf{X}\cdot\bf{X}}
$$


In [2]:
## question for developer:
# in the appendix of Pan2010, the flux term is multiplied by pstar but in lal it is multiplied by p. Why so?

def SEOBNR_RHS(t,var,m1,m2,EMGamma,tortoise):
    
    ## Unwrap all arguments
    eta = m1*m2/(m1+m2)/(m1+m2)
    q = np.array([var[0], var[1], var[2]])
    p = np.array([var[3], var[4], var[5]])
    S1 = np.array([var[6], var[7], var[8]])
    S2 = np.array([var[9], var[10], var[11]])
    
    ## Call necessary functions
    ham_and_potentials = hr.compute_Hreal_and_csi(m1,m2,EMGamma,tortoise,var[0],var[1],var[2],var[3],var[4],var[5],var[6],var[7],var[8],var[9],var[10],var[11])
    Hreal = ham_and_potentials[0]
    
    dHreal = dh.ham_first_derivs(m1,m2,tortoise,var[0],var[1],var[2],var[3],var[4],var[5],var[6],var[7],var[8],var[9],var[10],var[11])
    dHdX = np.array([dHreal[0],dHreal[1],dHreal[2]])/eta
    dHdP = np.array([dHreal[3],dHreal[4],dHreal[5]])/eta
    dHdS1 = np.array([dHreal[6],dHreal[7],dHreal[8]])
    dHdS2 = np.array([dHreal[9],dHreal[10],dHreal[11]])
    
    #for i in range(3):
    #    print("dHdP[%d] = %.16e"%(i,dHdP[i]))
    
    ## Compute R, R^-1 and R^-2
    
    R = np.linalg.norm(q)
    Rinv = 1/R
    R2inv = Rinv*Rinv
    
    ## Compute xi_a and xi_a - 1 and xi_a^-1
    
    xi_a = ham_and_potentials[1]
    xi_a_minusone = xi_a - 1
    xi_a_inv = 1/xi_a
    #print(xi_a)
    dxi_a = np.array([dHreal[12],dHreal[13],dHreal[14]])
    
    #print(dxi_a)
    ## Assign and Populate the Tortoise Matrix, its inverse, and its derivative
    
    kron_delta = lambda x,y: 1 if x== y else 0 
    
    T = np.zeros([3,3])
    Tinv = np.zeros([3,3])
    dT = np.zeros([3,3,3])
    
    ## T matrix has been validated
    for i in range(3):
        for j in range(3):
            T[i,j] = kron_delta(i,j) + q[i]*q[j]*xi_a_minusone*R2inv
            Tinv[i,j] = kron_delta(i,j) - q[i]*q[j]*xi_a_minusone*R2inv*xi_a_inv
    #for i in range(3):
    #    for j in range(3):
    #        print("T[%d,%d] = %.16e"%(i,j,T[i,j]))
            
    #for i in range(3):
    #    for j in range(3):
    #        print("Tinv[%d,%d] = %.16e"%(i,j,Tinv[i,j]))
                
    for i in range(3):
        for j in range(3):
            for k in range(3):
                dT[i,j,k] = ((kron_delta(i,k)*q[j] + q[i]*kron_delta(j,k) - 2*q[i]*q[j]*q[k]*R2inv)*xi_a_minusone + q[i]*q[j]*dxi_a[k])*R2inv 
    
    ## Compute the X derivatives
    
    dX = np.dot(T,dHdP)
    
    ## Compute Omega
    
    Omega = np.linalg.norm(np.cross(q,dX))*R2inv
    #print(Omega)
    
    ## Compute Angular Momentum and its magnitude, L
    
    Lvec = np.cross(q,p)
    L = np.linalg.norm(Lvec)
    
    ## Call the flux function to get dEdt
    
    dEdt = fl.compute_flux(m1,m2,EMGamma,tortoise,q,p,S1,S2,Omega,Hreal)
    print(t,dEdt)
    ## Compute the S1, S2 derivatives
    
    dS1 = np.cross(dHdS1,S1)
    dS2 = np.cross(dHdS2,S2)
    
    ## Compute the terms in the momentum derivative and compute the P derivative
    
    Ham_deriv_term_1 = np.zeros(3)
    
    for i in range(3):
        Ham_deriv_term_1[i] = 0 
        for j in range(3):
            Ham_deriv_term_1[i] += T[i,j]*(-dHdX[j])
    
    Ham_deriv_term_2 = np.zeros(3)
    ## no nested loop, define individual terms
    #for i in range(3):
    #    for j in range(3):
    #        for k in range(3):
    #            for l in range(3):
    #                for m in range(3):
    #                    Ham_deriv_term_2[i] += p[l]*Tinv[l,k]*dT[k,i,j]*T[j,m]*dHdP[m]
    
    Ham_deriv_term_2_T11 = np.zeros([3,3,3])
    Ham_deriv_term_2_T12 = np.zeros([3,3])
    Ham_deriv_term_2_T2 = np.zeros([3])
    
    for i in range(3):
        for j in range(3):
            for l in range(3):
                for k in range(3):
                    Ham_deriv_term_2_T11[i,j,l] += dT[i,k,j]*Tinv[k,l]
    
    for i in range(3):
        for j in range(3):
            for k in range(3):
                Ham_deriv_term_2_T12[i,j] += Ham_deriv_term_2_T11[i,j,k]*p[k]
    
    for i in range(3):
        for j in range(3):
            Ham_deriv_term_2_T2[i] += dHdP[j]*T[i,j]
    
    for i in range(3):
        for j in range(3):
            Ham_deriv_term_2[i] += Ham_deriv_term_2_T12[i,j]*Ham_deriv_term_2_T2[j]
                
    pstar = np.zeros(3)
    for i in range(3):
        for j in range(3):
            pstar[i] += T[i,j]*p[j]
    
    flux_term = -p*dEdt/Omega/L/eta
    
    #print("\n\n dHdX, T, Momentum Term 1:\n\n")
    #print(dHdX)
    #print(T)
    #print(Ham_deriv_term_1)
    #print("\n\n Tinv, dT, dHdP, Momentum Term 2:\n\n")
    #print(Tinv)
    #print("\n")
    #print(dT)
    #print("\n")
    #print(dHdP)
    #print(Ham_deriv_term_2)
    #print("\n\n DEdt, flux_term: \n \n")
    #print(dEdt/eta)
    #print(flux_term)
    
    
    dP = Ham_deriv_term_1 + Ham_deriv_term_2 + flux_term
    ## Return the values in the right order dX,dP,dS1,dS2
    
    return np.array([dX[0],dX[1],dX[2],dP[0],dP[1],dP[2],dS1[0],dS1[1],dS1[2],dS2[0],dS2[1],dS2[2]])


## Testing the RHS with the Initial Conditions

Here, we will plug into the RHS routine the initial conditions obtained from LALSuite in order to check if the function yields the correct right hand sides

In [3]:

#lal_init_params = {'m1': 2.3000000000000000e+01, 'm2': 2.3000000000000000e+01, 'x': 1.7078982385565073e+01, 'y': 0.0000000000000000e+00, 'z': 0.0000000000000000e+00, 'S1x':  2.500000000000000e-03, 'S1y': 0.0000000000000000e+00, 'S1z': 0.0000000000000000e+00, 'S2x': 2.500000000000000e-03, 'S2y': 0.000000000000000e+00, 'S2z': 0.000000000000000e+00, 'p1': -8.1282456846414118e-04, 'p2': 2.6586738983364794e-01, 'p3': 5.5013865451607893e-13,'tortoise': 1}
#m1 = lal_init_params['m1']
#m2 = lal_init_params['m2']
#eta = m1*m2/(m1+m2)/(m1+m2)
#Cartesianderivatives = np.array([-7.4616554087468274e-04,2.4313586897489273e-01,4.4408920985006262e-13,-3.7848674204967359e-03,-4.0697957718027191e-06,9.0867569024185775e-07,0.0000000000000000e+00,1.9427481667303873e-06,-1.9771221696866329e-13,0.0000000000000000e+00,1.9427481667303873e-06,-1.9771221696866329e-13,1.2566118227841438e-02,1.6698510755983208e-03])
#Cartesianderivativespert = np.array([-7.0691260531578426e-05,1.9874222685682041e-01,-9.4058094646243262e-10,-1.6544765874661142e-03,-2.3590158280943629e-07,2.4701751843257691e-07,0.0000000000000000e+00,4.8034009839754026e-07,-3.5478201337883784e-15,0.0000000000000000e+00,3.6994636188240312e-08,0.0000000000000000e+00,7.8325419818016818e-03,-2.7559458396515650e-06])
#MomentumTerm1 = [ -1.3088497326736192e-03 , -3.3404908468999864e-08 , 5.8397731095283234e-11 ]
#MomentumTerm2 = [ -4.4936867757153234e-05 , 3.3645070075027284e-08 , -9.2124371138343366e-18 ]
#FluxTerm = [ 3.2505033652576827e-10 , -4.4904787419519459e-07 , 1.6898324103064858e-14]
#dEdt = 2.7469013978683326e-07
#def SDA(x1,x2):
#    erel = abs((x1 - x2)/(0.5*(x1 + x2)))
#    return int(-np.log10(erel) + 1)



#EMGamma = 0.577215664901532860606512090082402431
#tortoise = lal_init_params['tortoise']
#lal_init_conditions = np.array([lal_init_params['x'], lal_init_params['y'], lal_init_params['z'], lal_init_params['p1'], lal_init_params['p2'], lal_init_params['p3'], lal_init_params['S1x'], lal_init_params['S1y'], lal_init_params['S1z'], lal_init_params['S2x'], lal_init_params['S2y'], lal_init_params['S2z']])
#rhs_first_step = SEOBNR_RHS(0,lal_init_conditions,m1,m2,EMGamma,tortoise)

#print("\n")
#for i in range(12):
#    print("dvalues[%d] (lal, nrpy, sda,allowed) = %.16e , %.16e , %d , %d"%(i,Cartesianderivatives[i],rhs_first_step[i],SDA(Cartesianderivatives[i],rhs_first_step[i]),SDA(Cartesianderivatives[i],Cartesianderivativespert[i])))


In [4]:
## print out tortoise to make sure all inputs are consistent
## toggle the inputs to see which terms are problematic

In [6]:
## simply ported final initial conditions from our lal solver

EMGamma = 0.577215664901532860606512090082402431
var_init = np.array([ 1.7077661371325462e+01, 0.0000000000000000e+00, 0.0000000000000000e+00, -7.1946742122340718e-04, 2.6577885496497711e-01, 4.0197262449583424e-18, 0.0000000000000000e+00, 0.0000000000000000e+00, 2.5000000000000001e-03, 0.0000000000000000e+00, 0.0000000000000000e+00, 2.5000000000000001e-03])
params = (23,23,EMGamma,1)
m1 = params[0]
m2 = params[1]
eta = m1*m2/(m1+m2)/(m1+m2)
max_time = 1.

#result = spint.solve_ivp(SEOBNR_RHS,(0,max_time),var_init,method = 'RK45',dense_output=True,args = params)

result = SEOBNR_RHS(0,var_init,m1,m2,EMGamma,1)
dt = 1
time_series = np.arange(0.,max_time,dt)
#print(time_series)

sparse_array = result.y
dense_sol = result.sol.__call__(time_series)

plot_array = np.array([dense_sol[:,i] for i in range(len(time_series))])

plt.plot(plot_array[:,0],plot_array[:,1])
plt.axis('square')
plt.savefig('inspiraltrajectoryv4p.png',dpi = 300)


2 1 nan
2 2 0.0880142491682515
3 1 nan
3 2 0.0003880549929822062
3 3 nan
4 1 nan
4 2 -4.164870533806282e-05
4 3 nan
4 4 0.0009247801850125403
5 1 -0.0
5 2 -1.7531038256434024e-07
5 3 0.0
5 4 5.199903813737008e-06
5 5 -0.0
6 1 0.0
6 2 1.2952346746387594e-08
6 3 -0.0
6 4 -9.684752007630265e-07
6 5 0.0
6 6 1.5655985125143117e-05
7 1 -0.0
7 2 4.450567207916154e-11
7 3 0.0
7 4 -5.797505123434854e-09
7 5 -0.0
7 6 9.087588897258917e-08
7 7 0.0
8 1 -0.0
8 2 -2.4718868940993363e-12
8 3 0.0
8 4 7.182642056202686e-10
8 5 -0.0
8 6 -2.2022264230679677e-08
8 7 0.0
8 8 3.2172065642733267e-07
0 nan


AttributeError: 'numpy.ndarray' object has no attribute 'y'

In [ ]:
finstate = np.array([dense_sol[0,-1],dense_sol[1,-1],dense_sol[2,-1],dense_sol[3,-1],dense_sol[4,-1],dense_sol[5,-1],dense_sol[6,-1],dense_sol[7,-1],dense_sol[8,-1],dense_sol[9,-1],dense_sol[10,-1],dense_sol[11,-1]])
print(finstate)

In [ ]:
# get waveform amplitudes

amplitudes = []
for i in range(len(dense_sol[0])):
    #eta = params[0]*params[1]/(params[0] + params[1])/(params[0] + params[1])
    q = np.array([dense_sol[0,i],dense_sol[1,i],dense_sol[2,i]])
    p = np.array([dense_sol[3,i],dense_sol[4,i],dense_sol[5,i]])
    S1 = np.array([dense_sol[6,i],dense_sol[7,i],dense_sol[8,i]])
    S2 = np.array([dense_sol[9,i],dense_sol[10,i],dense_sol[11,i]])
    ham_and_potentials = hr.compute_Hreal_and_csi(m1,m2,EMGamma,tortoise,q[0],q[1],q[2],p[0],p[1],p[2],S1[0],S1[1],S1[2],S2[0],S2[1],S2[2])
    Hreal = ham_and_potentials[0]
    
    dHreal = dh.ham_first_derivs(m1,m2,tortoise,q[0],q[1],q[2],p[0],p[1],p[2],S1[0],S1[1],S1[2],S2[0],S2[1],S2[2])
    dHdP = np.array([dHreal[3],dHreal[4],dHreal[5]])/eta
    
    #for i in range(3):
    #    print("dHdP[%d] = %.16e"%(i,dHdP[i]))
    
    ## Compute R, R^-1 and R^-2
    
    R = np.linalg.norm(q)
    Rinv = 1/R
    R2inv = Rinv*Rinv
    
    ## Compute xi_a and xi_a - 1 and xi_a^-1
    
    xi_a = ham_and_potentials[1]
    xi_a_minusone = xi_a - 1
    xi_a_inv = 1/xi_a
    #print(xi_a)
    dxi_a = np.array([dHreal[12],dHreal[13],dHreal[14]])
    
    #print(dxi_a)
    ## Assign and Populate the Tortoise Matrix, its inverse, and its derivative
    
    kron_delta = lambda x,y: 1 if x== y else 0 
    
    T = np.zeros([3,3])
    
    ## T matrix has been validated
    for i in range(3):
        for j in range(3):
            T[i,j] = kron_delta(i,j) + q[i]*q[j]*xi_a_minusone*R2inv
    
    ## Compute the X derivatives
    
    dX = np.dot(T,dHdP)
    
    ## Compute Omega
    
    Omega = np.linalg.norm(np.cross(q,dX))*R2inv
    v = np.cbrt(Omega)
    rcrossp = np.cross(q,p)
    rcrosspmag = np.linalg.norm(rcrossp)
    m1hat = m1/(m1+m2)
    m2hat = m2/(m1+m2)
    S1_over_m12 = S1/m1hat/m1hat
    S2_over_m22 = S2/m2hat/m2hat
    Lhat = rcrossp/rcrosspmag
    s1dotL = np.dot(S1_over_m12,Lhat)
    s2dotL = np.dot(S2_over_m22,Lhat)
    
    #print("s1dotL = ",s1dotL)
    #print("s2dotL = ",s2dotL)
    
    chiS = 0.5*(s1dotL + s2dotL)
    chiA = 0.5*(s1dotL - s2dotL)
    amplitudes.append(wf.compute_hFlm_for_flux(params[0], params[1], params[2], params[3], q, p, S1, S2, 2, 2, Hreal, v,chiA,chiS))



In [ ]:
plt.plot(time_series[0:200],amplitudes[0:200])

data = np.loadtxt("outputv4Pindex0.txt")
M = m1+m2
t_M = M*4.93e-6
t_data = (data[:,0] - data[0,0])/t_M
print(t_data)
print(time_series)
amp = np.array([np.sqrt(data[i,1]**2 + data[i,2]**2) for i in range(len(data))])
amp *= amplitudes[0]/amp[0]
plt.plot(t_data[0:50],amp[0:50])

In [ ]:
data = "outputv4Pindex0.txt"
datapert = "outputv4Pindex0pert.txt"
LALvalues = []
LALvaluespert = []
eta = (m1*m2)/(m1+m2)/(m1+m2)

with open(data) as file, open(datapert) as filepert:
    data_lines = file.readlines()
    pertdata_lines = filepert.readlines()
    for i in range(len(data_lines)//2):
        valuepairs = data_lines[2*i].strip('{}\n').split(',')
        pertvaluepairs = pertdata_lines[2*i].strip('{}\n').split(',')
        values = dict()
        pertvalues = dict()
        #print(i)
        for j in range(len(valuepairs)):
            pair = valuepairs[j].split(': ')
            pertpair = pertvaluepairs[j].split(': ')
            #print(pertpair)
            values[pair[0].strip(" ''")] = float(pair[1])
            pertvalues[pertpair[0].strip(" ''")] = float(pertpair[1])
        LALvalues.append([values['x'],values['y']])
        LALvaluespert.append([pertvalues['x'],pertvalues['y']])


plt.plot(plot_array[:,0],plot_array[:,1])
LALvalues = np.array(LALvalues)
LALvaluespert = np.array(LALvaluespert)
plt.plot(LALvalues[:,0],LALvalues[:,1])
plt.plot(LALvaluespert[:,0],LALvaluespert[:,1])
plt.axis('square')




In [ ]:
plt.plot(plot_array[:,0],plot_array[:,1])
dt = 1
time_series = np.arange(0.,max_time,dt)
print(time_series)
sparse_array = result.y
dense_sol = result.sol.__call__(time_series)

plot_array = np.array([dense_sol[:,i] for i in range(len(time_series))])

LALvalues = np.array(LALvalues)
plt.plot(np.log10(np.abs((LALvalues[:,0] - plot_array[:,0])/LALvalues[:,0])))
plt.plot(np.log10(np.abs((LALvaluespert[:,0]- LALvalues[:,0])/LALvalues[:,0])))
plt.plot(LALvalues[:,0])
plt.plot(plot_array[:,0])

In [ ]:
plt.plot(LALvalues[:,1])
plt.plot(plot_array[:,1])

In [ ]:
plt.plot([np.linalg.norm([plot_array[i,0],plot_array[i,1]]) for i in range(len(plot_array))])
plt.plot([np.linalg.norm([LALvalues[i,0],LALvalues[i,1]]) for i in range(len(plot_array))])
